In [2]:
import sys
sys.executable
sys.path.append('/home/sai/miniconda3/envs/depth_model/bin/python')
sys.executable
import numpy as np
import pandas as pd
from random import random
from numpy import array
from numpy import cumsum
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import TimeDistributed

ImportError: Keras requires TensorFlow 2.2 or higher. Install TensorFlow via `pip install tensorflow`

In [2]:
bool_array_for=[0]*11
prev_correct_point=-1
def first_check(probs,labels,x):
  global bool_array_for
  bool_array_for=[0]*11
  i=1
  for l in range(len(probs)):
    if(probs[l]==1):
        bool_array_for[l]=labels[l]
        i=l+1
    else:
        break
  x=list(x.reshape(1,len(x))[0])
  #x=list(noise_data['percent'])
  #y=list(noise_data['date'])
  if(bool_array_for[i-1]==2 and prev_correct_point!=-1):
        x[i-1]=prev_correct_point
  j=1
  # 0 for normal
  # 1 for refill
  # 2 for noise
  # abs(x[i]-x[i-1])>=2 This is for checking if its peak or just x[i] ~ x[i-1]
  while(i<(len(x)-6)):
    if( (x[i]>x[i-1] ) and abs(x[i+1]-x[i-1])<=2  and abs(x[i]-x[i-1])>=5 ) : 
        bool_array_for[j]=2
        x.pop(i)
        #y.pop(i)
        i-=1
        
    elif(  (x[i]<x[i-1] ) and abs(x[i+1]-x[i-1])<=2  and abs(x[i]-x[i-1])>=5 ) : 
        bool_array_for[j]=2
        x.pop(i)
        #y.pop(i)
        i-=1
        
    elif(  x[i]>x[i-1] and  abs(x[i+2]-x[i-1])<=2 and abs(x[i]-x[i-1])>=5  ) : 
        bool_array_for[j]=2
        x.pop(i)
        #y.pop(i)
        i-=1
        
    elif(  x[i]<x[i-1] and  abs(x[i+2]-x[i-1])<=2 and abs(x[i]-x[i-1])>=5  ) : 
        bool_array_for[j]=2
        x.pop(i)
        #y.pop(i)
        i-=1
        #continue
 
    elif(  x[i]>x[i-1] and  abs(x[i+3]-x[i-1] <=2) and abs(x[i]-x[i-1])>=5) : 
        bool_array_for[j]=2
        x.pop(i)
        #y.pop(i)
        i-=1
        #continue
    elif( x[i]<x[i-1] and abs(x[i+3]-x[i-1] <=2) and abs(x[i]-x[i-1])>=5) : 
        bool_array_for[j]=2
        x.pop(i)
        #y.pop(i)
        i-=1
        #continue
    elif(  x[i]<x[i-1] and  abs(x[i+4]-x[i-1])<=2 and abs(x[i]-x[i-1])>=5) : 
        bool_array_for[j]=2
        x.pop(i)
        #y.pop(i)
        i-=1
        #continue
    elif(  x[i]>x[i-1] and  abs(x[i+4]-x[i-1])<=2 and abs(x[i]-x[i-1])>=5) : 
        bool_array_for[j]=2
        x.pop(i)
        #y.pop(i)
        i-=1
        #continue
    
    
    elif(  x[i]>x[i-1] and abs(x[i+5]-x[i-1])<=2 and abs(x[i]-x[i-1])>=5) : 
        bool_array_for[j]=2
        x.pop(i)
        #y.pop(i)
        i-=1
        #continue
    elif(  x[i]<x[i-1] and  abs(x[i+5]-x[i-1])<=2 and abs(x[i]-x[i-1])>=5) : 
        bool_array_for[j]=2
        x.pop(i)
        #y.pop(i)
        i-=1
        #continue    
    elif(  x[i]>x[i-1] and abs(x[i+6]-x[i-1])<=2 and abs(x[i]-x[i-1])>=5) : 
        bool_array_for[j]=2
        x.pop(i)
        #y.pop(i)
        i-=1
        #continue
    elif(  x[i]<x[i-1] and abs(x[i+6]-x[i-1])<=2 and abs(x[i]-x[i-1])>=5) : 
        bool_array_for[j]=2
        x.pop(i)
        #y.pop(i)
        i-=1
        #continue    
#     elif((len(x)-i)>=7 and x[i]>x[i-1] and abs(x[i+7]-x[i-1])<=2 and abs(x[i]-x[i-1])>=5):
#          bool_array_for[j]=2
#          x.pop(i)
#          i-=1
        
#     elif( (len(x)-i)>=7 and x[i]<x[i-1] and abs(x[i+7]-x[i-1])<=2 and abs(x[i]-x[i-1])>=5) : 
#          bool_array_for[j]=2
#          x.pop(i)
#          i-=1
        
    elif(x[i]-x[i-1]>5):
        bool_array_for[j]=1
    
    
        
    i+=1    
    j+=1
    print(bool_array_for)
    

In [3]:
from keras.models import model_from_json

json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
#yhat = loaded_model.predict_classes(X_test, verbose=0)
#for i in range(n_timesteps):
#    print('Expected:',Y_test[0, i], 'Predicted', yhat[0, i])

Loaded model from disk


In [4]:
prev_pred=[[0 ,0, 0]]*11
count=[[0 ,0, 0]]*11
def prediction(probs,labels,percent):
    global prev_correct_point
    x_old=[i for i in percent]
    for i in range(10):
        prev_pred[i]=prev_pred[i+1]
        count[i]=count[i+1]
    prev_pred[10]=[0, 0, 0]
    count[10]=[0, 0, 0]
    model_classes=loaded_model.predict_classes(percent.reshape(1,11,1), verbose=0)[0]
    model_pred=loaded_model.predict(percent.reshape(1,11,1), verbose=0)[0]
    count[10][model_classes[10]]+=1
    first_check(probs,labels,percent)
    new_labels=[]
    new_probs=[]
    
    for i in range(len(x_old)):
        prev_pred[i][model_classes[i]]=((prev_pred[i][model_classes[i]])*(count[i][model_classes[i]])+model_pred[i][model_classes[i]])/(count[i][model_classes[i]]+1)
        count[i][model_classes[i]]+=1
        sumi=count[i][0]+count[i][1]+count[i][2]
        final_0=((model_pred[i][0]))
        final_1=((model_pred[i][1]))
        final_2=((model_pred[i][2]))
        print(prev_pred[i][0])
        if(bool_array_for[i] !=0):
         if(i<5):
             new_labels.append(bool_array_for[i])
             new_probs.append(1)
             if(bool_array_for[i]==1):
                    prev_correct_point=x_old[i]
         else: 
          if(final_0>final_1):
                if(final_0>final_2):
                    new_labels.append(0)
                else:
                    new_labels.append(2)
          else:
            if(final_1>final_2):
                new_labels.append(1) 
            else:
                new_labels.append(2)
          new_probs.append(prev_pred[i][new_labels[i]])
        if(bool_array_for[i]==0):
            if(i<5):
             new_labels.append(0)
             new_probs.append(1)
             prev_correct_point=x_old[i]
            else:
             if(final_0>final_1):
                if(final_0>final_2):
                    new_labels.append(0)
                else:
                    new_labels.append(2)
             else:
               if(final_1>final_2):
                new_labels.append(1) 
               else:
                new_labels.append(2)
             new_probs.append(prev_pred[i][new_labels[i]])
    return (new_labels,new_probs)                

In [5]:
df=pd.read_csv('Synthetic_Train_Data.csv')
df=df.drop(columns=['Unnamed: 0.1'])

In [6]:
X=[]
for i in range(df.shape[0]):
    X.append([])
    for j in range(11):
        X[i].append(df[df.columns[j+1]][i])
Y=[]
for i in range(df.shape[0]):
    Y.append([])
    for j in range(11):
        Y[i].append(df[df.columns[11+j+1]][i])

In [7]:
for i in range(len(Y[0])):
    for j in range(len(Y)):
        if(Y[j][i] ==0):
            Y[j][i]=[1,0,0]
        elif(Y[j][i] ==1):
            Y[j][i]=[0,1,0]
        elif(Y[j][i] ==2):
            Y[j][i]=[0,0,1]

In [8]:
X=np.array(X)
X=X.reshape(X.shape[0],X.shape[1],1)
Y=np.array(Y)

In [9]:
probs=[0]*11
labels=[0]*11


In [13]:
print(prediction(probs,labels,X[201]))
print(bool_array_for)

[0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0]
0.9970551869448494
0.9970551869448494
0.9970551869448494
0.9971871144241757
0.9973351610334296
0.997468301653862
0.9975888076282683
0.9976983747699044
0.7488494515419006
0.6538743575414022
0.49978193640708923
([0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 0.997468301653862, 0.9975888076282683, 0.9976983747699044, 0.7488494515419006, 0.6538743575414022, 0.49978193640708923])
[0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0]


array([[93.],
       [93.],
       [36.],
       [37.],
       [37.],
       [37.],
       [37.],
       [37.],
       [37.],
       [37.],
       [37.]])

In [39]:
prev_pred=[[0 ,0, 0]]*11

In [43]:
x_old=[i for i in X[10]]

In [45]:
len(x_old)

11

In [89]:
bool_array_for

[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]

In [27]:
json_input=pd.read_json('input.json')

In [32]:
json_input

,id,date,percent,label,probability
0,9,2020-01-10 16:47:08+00:00,27,0,0
1,10,2020-01-10 18:47:13+00:00,26,0,0
2,11,2020-01-10 20:47:10+00:00,26,0,0
3,12,2020-01-10 22:47:07+00:00,26,0,0
4,13,2020-01-11 00:47:07+00:00,27,0,0
5,14,2020-01-11 04:47:07+00:00,26,0,0
6,15,2020-01-11 05:47:12+00:00,26,0,0
7,16,2020-01-11 07:47:07+00:00,26,0,0
8,17,2020-01-11 08:47:10+00:00,26,0,0
9,18,2020-01-11 11:47:07+00:00,26,0,0


In [38]:
def json_file_export(json_input):
   probs=list(json_input['probability']) 
   labels=list(json_input['label'])
   percent=list(json_input['percent']) 
   new_labels,new_probs=prediction(probs,labels,np.array(percent))
   json_input['probability']=new_probs
   json_input['label']=new_labels
   json_input.to_json("result.json")  

In [39]:
json_file_export(json_input)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.9977577976558519
0.9977574224273363
0.9978464865684509
0.9979290847594922
0.9980050965591714
0.9980761642966952
0.7990774393081665
0.7404043674468994
0.6665199001630148
0.999998927116394
0.4999518394470215
